It's interesting to note that in deep learning, the evolution of architecture and techniques is very rapid, but that certain aspects can remain present for years in bits of code, without really knowing if they are totally necessary. Most of the time, we take for granted that these were optimized and this should not be taken care of.

One such aspect is *input normalization*, which is used as input to most convolutional networks used to process visual images. This is all the more true as we often use in the community networks that are pre-trained for images normalized to given values, like VGG or ResNET.

**I'm interested here in whether these networks remain effective when we change the input normalization.** Indeed, as the weights of the first convolutional layer are learned, and as convolution kernels can be multiplied by arbitrary values thanks to the bias in the convolutions, normalizing to a certain value makes no sense, and simply normalizing does. This is even more evident when that values are given with 3 digits of precision! I propose here to show quantitatively that this is the case, first for a *historical* network [LeNet](https://en.wikipedia.org/wiki/LeNet) applied to the MNIST challenge, and then for ResNet applied to ImageNet.


<!-- TEASER_END -->

Let's first initialize the notebook:

In [1]:
from __future__ import division, print_function
import numpy as np
np.set_printoptions(precision=6, suppress=True)
import os
%matplotlib inline
#%config InlineBackend.figure_format='retina'
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
phi = (np.sqrt(5)+1)/2
fig_width = 10
figsize = (fig_width, fig_width/phi)
from IPython.display import display, HTML
def show_video(filename): 
    return HTML(data='<video src="{}" loop autoplay width="600" height="600"></video>'.format(filename))
%load_ext autoreload
%autoreload 2

# The MNIST challenge and the *French Touche* 

First, let's explore the *LeNet* network  (Lecun, Y.; Bottou, L.; Bengio, Y.; Haffner, P. (1998). "Gradient-based learning applied to document recognition" (PDF). Proceedings of the IEEE. 86 (11): 2278–2324. doi:10.1109/5.726791. S2CID 14542261), whose objective is to categorize images of written digits, one of the first great successes of Multilayer neural networks. For this one, we're going to use the classic implementation, as proposed in the PyTorch library example series. 

The cell below allows you to do everything: first load the libraries, then define the neural network, and finally define the learning and testing procedures that are applied to the database. The output Accura value corresponds to the percentage of letters that are correctly classified.

In this cell, we'll isolate the two parameters used to set the mean and standard deviation applied to the normalization function, which we'll be manipulating in the course of this book.

In [2]:
# adapted from https://raw.githubusercontent.com/pytorch/examples/refs/heads/main/mnist/main.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

epochs = 2

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if verbose and batch_idx % log_interval  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader, verbose=True):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    if verbose:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct / len(test_loader.dataset)


def main(mean, std, epochs=epochs, log_interval=100, verbose=True):
    # Training settings
    use_cuda = torch.cuda.is_available()
    use_mps = torch.backends.mps.is_available()

    torch.manual_seed(1998) # FOOTIX rules !

    if use_cuda:
        device = torch.device("cuda")
    elif use_mps:
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': 64}
    test_kwargs = {'batch_size': 1000}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((mean,), (std,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)

    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=1.0)

    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
    for epoch in range(1, epochs):
        train(model, device, train_loader, optimizer, epoch, log_interval=log_interval, verbose=verbose)
        scheduler.step()

    accuracy = test(model, device, test_loader, verbose=verbose)
    return accuracy



Now that we've defined the entire protocol, we can test it in its most classic form, as delivered in the original code with 3 digits precision (!) :

In [3]:
accuracy = main(mean=0.1307, std=0.3081)
print(f'{accuracy=:.4f}')


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304859
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.289305
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.213311
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.308440
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.147784
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.176286
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.155225
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.146125
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.139085
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.113915

Test set: Average loss: 0.0500, Accuracy: 9830/10000 (98%)

accuracy=0.9830


This was first introduced in that model on [Jan 17, 2017](https://github.com/pytorch/examples/commit/32c7386aef93737926069ee284d827f8e954e086) and these excat values have not changed since.

One advantage of our code is that we can now manipulate these two values, to see if starting from the same code we obtain an accuracy value that is different.



In [4]:
accuracy = main(mean=0., std=1., epochs=epochs, verbose=False)
print(f'{accuracy=:.4f}')


accuracy=0.9796


The result seems similar, but it's not enough to demonstrate that the mean and standard deviation have no effect on learning.

## using optuna

I'm now going to go the whole hog, using a library that allows me to test multiple values and thus optimize the parameters. If mean and deviation are so important, then we'll converge on a fixed set of values, whereas if it's less important, the values may be quite scattered. In particular, this will allow us to choose an arbitrary value, such as a mean of zero and a standard deviation of 1, better known as the *normal* normalization.

In [5]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
path_save_optuna =  'optuna-MNIST.sqlite3'

In [6]:
%rm {path_save_optuna}

In [ ]:
def objective(trial):
    mean = trial.suggest_float('mean', -10, 10, log=False)
    std = trial.suggest_float('std', 0.1, 10, log=True)
    accuracy = main(mean=mean, std=std, epochs=epochs, verbose=False)
    return accuracy


if not(os.path.isfile(path_save_optuna)):
    study = optuna.create_study(direction='maximize', load_if_exists=True, 
                                storage=f"sqlite:///{path_save_optuna}", study_name='MNIST')
    study.optimize(objective, n_trials=200, n_jobs=1, show_progress_bar=True)
    print(50*'-.')
    print("Best params: ", study.best_params)
    print("Best value: ", study.best_value)
    print("Best Trial: ", study.best_trial)
    print(50*'=')


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
# https://optuna.readthedocs.io/en/stable/reference/visualization/matplotlib/generated/optuna.visualization.matplotlib.contour.html
from optuna.visualization.matplotlib import plot_contour

fig = plot_contour(study, params=["mean", "std"], target_name="Accuracy")
fig.set_size_inches(figsize)
fig

## some book keeping for the notebook

In [ ]:
%load_ext watermark
%watermark -i -h -m -v -p numpy,matplotlib,torch  -r -g -b